In [7]:
base_url = "https://raw-data-api0.hotosm.org/v1"
headers = {"accept": "application/json", "Content-Type": "application/json"}

In [8]:
query_url = f"{base_url}/snapshot/"
query_url

'https://raw-data-api0.hotosm.org/v1/snapshot/'

In [79]:
import requests
import json
import zipfile
from io import BytesIO
import numpy as np
import pandas as pd
from shapely.geometry import shape

In [ ]:
pip install folium

In [230]:
import folium

In [231]:
geom={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              83.46315999320524,
              27.700092338830743
            ],
            [
              83.46195891081277,
              27.69889579587334
            ],
            [
              83.46194048799703,
              27.695821014347786
            ],
            [
              83.46348800450971,
              27.695592645155344
            ],
            [
              83.46517369214064,
              27.6955273967272
            ],
            [
              83.46484568083457,
              27.697792403005664
            ],
            [
              83.46432984199652,
              27.69935832202509
            ],
            [
              83.46315999320524,
              27.700092338830743
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [232]:
boundary=shape(geom["features"][0]["geometry"])
minx, miny, maxx, maxy = boundary.bounds

In [233]:
m=folium.Map(zoom_start=10, control_scale=True)
m.fit_bounds([[maxy,maxx],[miny,minx]])
m.add_child(folium.GeoJson(data = geom))
m

In [243]:
query={"filters":{
    "tags": {
        "all_geometry": {
            "join_or": {
                # Use OSM Tags here
                "building":[],
                "highway": [],
                "waterway":[]
                
            }
        }
    }
},
"geometryType": [
    "polygon",
    "line",
    "line"
],
"centroid": "false"
}
query["geometry"] = geom["features"][0]["geometry"]
query

{'filters': {'tags': {'all_geometry': {'join_or': {'building': [],
     'highway': [],
     'waterway': []}}}},
 'geometryType': ['polygon', 'line', 'line'],
 'centroid': 'false',
 'geometry': {'coordinates': [[[83.46315999320524, 27.700092338830743],
    [83.46195891081277, 27.69889579587334],
    [83.46194048799703, 27.695821014347786],
    [83.46348800450971, 27.695592645155344],
    [83.46517369214064, 27.6955273967272],
    [83.46484568083457, 27.697792403005664],
    [83.46432984199652, 27.69935832202509],
    [83.46315999320524, 27.700092338830743]]],
  'type': 'Polygon'}}

In [244]:
result = requests.post(query_url, data=json.dumps(query), headers=headers)
result.status_code

200

In [245]:
if result.status_code == 200:
    task_id = result.json()['task_id']
    print(task_id)
else:
    print(result)

c2d38e23-2f39-44bb-a207-f385bbe5b7ab


In [248]:
task_url = f"{base_url}/tasks/status/{task_id}"
# extracts = requests.get(task_url)
while True:
    result = requests.get(task_url, headers=headers)
    if result.json()['status'] == "PENDING":
        time.sleep(1)
    elif result.json()['status'] == "SUCCESS":
        break

In [249]:
zip_url = result.json()['result']['download_url']
zip_url
result = requests.get(zip_url, headers=headers)
# result.content
fp = BytesIO(result.content)
zfp = zipfile.ZipFile(fp, "r")
zfp.extract("Export.geojson", "/tmp/")
data = eval(zfp.read("Export.geojson"))


In [250]:
m.add_child(folium.GeoJson(data = data))